In [166]:
import turicreate

In [167]:
products = turicreate.SFrame('product review')
products.head(3)

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0


# creating a column for word count that includes all words

In [168]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

# including only selected words

# adding column 'awesome' as example

In [61]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [170]:
products['awesome'] = products['word_count'].apply(lambda x: x['awesome'] if 'awesome' in x.keys() else 0)

In [171]:
products['awesome']

dtype: float
Rows: 183531
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ... ]

# adding columns for other words except awesome in one go

In [172]:
selected_word = ['great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [173]:
for word in selected_word:
    products[word] = products['word_count'].apply(lambda x: x[word] if word in x.keys() else 0)


In [174]:
products.head(2)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	float
	terrible	int
	awful	int
	wow	int
	hate	int

Rows: 2

Data:
+--------------------------+-------------------------------+--------+
|           name           |             review            | rating |
+--------------------------+-------------------------------+--------+
| Planetwise Flannel Wipes | These flannel wipes are OK... |  3.0   |
|  Planetwise Wipe Pouch   | it came early and was not ... |  5.0   |
+--------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'handles': 1.0, 'strippin... |   0.0   |  0.0  |    0.0    |   0.0   | 0.0  |
| {'recommend': 1.0, 'highly... |   0.0   |  0.0  |    0.0    |   0.0   | 1.0  |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+
| horrible | bad | terrible | awful | wow | hate |
+----------+-----+----------+-------+-----+------+
|    0     | 0.0 |    0     |   0   |  0  |  0   |
|    0     | 0.0 |    0     |   0   |  0  |  0   |
+----------+-----+----------+-------+-----+------+
[2 rows x 15 columns]

# Counting the number of times each selected word is used in the dataset

In [175]:
counts= []
counts = [(word, products[word].sum()) for word in selected_words]
counts

[('awesome', 4075.0),
 ('great', 59536.0),
 ('fantastic', 1765.0),
 ('amazing', 2726.0),
 ('love', 43867.0),
 ('horrible', 1245),
 ('bad', 4950.0),
 ('terrible', 1282),
 ('awful', 753),
 ('wow', 461),
 ('hate', 1285)]

# word occuring most no of times

In [176]:
sorted(counts, key = lambda x: x[1])[-1]

('great', 59536.0)

# word occuring least number of times

In [177]:
sorted(counts, key = lambda x: x[1])[0]

('wow', 461)

# building the classifier

In [178]:
products = products[products['rating']!= 3]

In [179]:
products['sentiment']= products['rating'] >= 4

In [180]:
train_data,test_data = products.random_split(.8, seed=0)

In [181]:
selected_words_model = turicreate.logistic_classifier.create(train_data, target='sentiment', features = selected_words)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 126775

Number of classes           : 2

...

| 4         | 5        | 0.381296     | 0.847935          | 0.841600            |

| 5         | 6        | 0.472144     | 0.847935          | 0.841600            |

| 6         | 7        | 0.563340     | 0.847935          | 0.841600            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

# Examining weight of words

In [182]:
coeffs = selected_words_model.coefficients.sort('value')
coeffs

name,index,class,value,stderr
horrible,None,1,-2.2334023738795734,0.0816101858730564
terrible,None,1,-2.22392818030467,0.07932194524872109
awful,None,1,-2.0230888941456264,0.103469668205189
hate,None,1,-1.3384713597582658,0.07967890499477706
bad,None,1,-0.9950132516470084,0.03942749534122459
wow,None,1,0.014741522163067941,0.16374142730807514
great,None,1,0.8549450576873948,0.019382711365510737
fantastic,None,1,0.9011730684283838,0.11629911237951046
amazing,None,1,1.1222432282417314,0.10280264233699381
awesome,None,1,1.1346173462915963,0.08608887060492235


# most weight

In [183]:
coeffs[-1]

{'name': 'love',
 'index': None,
 'class': 1,
 'value': 1.355846297688754,
 'stderr': 0.028802090330117172}

# least weight

In [184]:
coeffs[0]

{'name': 'horrible',
 'index': None,
 'class': 1,
 'value': -2.2334023738795734,
 'stderr': 0.0816101858730564}

# evaluate the model

In [185]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8461145808311314,
 'auc': 0.6928533458215148,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  115  |
 |      0       |        0        |  318  |
 |      0       |        1        |  5010 |
 |      1       |        1        | 27861 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157723470343649,
 'log_loss': 0.3962565290359168,
 'precision': 0.8475860180706398,
 'recall': 0.9958893337146125,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 532

# create all words model

In [186]:
train_data_all, test_data_all = products.random_split(0.8,seed=0)
sentiment_model = turicreate.logistic_classifier.create(train_data_all,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 2.203572     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 2.523639     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 2.993633     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 4.706981     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [187]:
sentiment_model.evaluate(test_data)

{'accuracy': 0.9176975738650012,
 'auc': 0.9342357833151299,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  1397 |
 |      1       |        0        |  1344 |
 |      0       |        0        |  3931 |
 |      1       |        1        | 26632 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.951057941255245,
 'log_loss': 0.3304787187241236,
 'precision': 0.9501587641371436,
 'recall': 0.9519588218472976,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+--------------------+-------+------+
 | threshold |        fpr         |        tpr         |   p   |  n   |
 +-----------+--------------------+--------------------+-------+------+
 |    0.0

accuracy for selected words model is 0.846
accuracy for model with all words is 0.917
so our accuracy has decreased

now, we can calculate the majority classifier accuracy

In [188]:
products['sentiment'].show()

Materializing SArray

In [189]:
products.shape[0]

166752

we can see that most common class is sentiment = 1

In [190]:
products[newproducts['sentiment']==1].shape[0]

140259

In [191]:
print("Majority class accuracuy = " + str(140259/166752))

Majority class accuracuy = 0.8411233448474381


Thus, the selected words classifier accuracy is only slightly better than majority class accuracy. Using all words gives much better accuracy than both methods.

# Investigate why all words count gives better than selected words count accuracy

this involves looking at a popular product "baby diaper champ"

# calculating predicted sentiment for this diaper product with all words model

In [231]:
diaper_champ_reviews = products[products['name']=='Baby Trend Diaper Champ']
diaper_champ_reviews.head(3)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	awesome	float
	great	float
	fantastic	float
	amazing	float
	love	float
	horrible	int
	bad	float
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int

Rows: 3

Data:
+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Ok - newsflash.  Diapers a... |  4.0   |
| Baby Trend Diaper Champ | My husband and I selected ... |  1.0   |
| Baby Trend Diaper Champ | Excellent diaper disposal ... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+---------+-------+-----------+---------+------+
|           word_count          | awesome | great | fantastic | amazing | love |
+-------------------------------+---------+-------+-----------+---------+------+
| {'convenient': 1.0, 'more'... |   0.0   |  0.0  |    0.0    |   0.0   | 0.0  |
| {'system': 1.0, 'try': 1.0... |   0.0   |  0.0  |    0.0    |   0.0   | 0.0  |
| {'nose': 1.0, 'for': 2.0, ... |   0.0   |  0.0  |    0.0    |   0.0   | 0.0  |
+-------------------------------+---------+-------+-----------+---------+------+
+----------+-----+----------+-------+-----+------+-----------+
| horrible | bad | terrible | awful | wow | hate | sentiment |
+----------+-----+----------+-------+-----+------+-----------+
|    0     | 0.0 |    0     |   0   |  0  |  0   |     1     |
|    0     | 0.0 |    0     |   0   |  0  |  0   |     0     |
|    0     | 0.0 |    0     |   0   |  0  |  0   |     1     |
+----------+-----+----------+-------+-----+------+-----------+
[3 rows x 16 columns]

In [232]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type = 'probability')

most positive review

In [233]:
diaper_champ_reviews.sort('predicted_sentiment', ascending=False)[0]

{'name': 'Baby Trend Diaper Champ',
 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.",
 'rating': 4.0,
 'word_count': {'key': 1.0,
  'have': 1.0,
  'pieces': 1.0,
  'between': 1.0,
  'cushioned': 1.0,
  'piece': 1.0,
  'take': 1.0,
  'overloading': 1.0,
  'be': 1.0,
  'looking': 1.0,
  're': 1.0,
  'stinkiness': 1.0,
  'retain': 1.0,
  'now': 1.0,
  'wonderful': 1.0,
  'worked': 1.0,
  '16': 1.0,
  'and': 3.0,
  'months': 1.0,
  've': 1.0,
  'in': 1.0,
  'i': 3.0,
  'experienced': 1.0,
  'read': 1.0,
 

# calculating predicted sentiment for this diaper product with selected words model

In [234]:
diaper_champ_reviews['predicted_sentiment_sel_words'] = selected_words_model.predict(diaper_champ_reviews, output_type = 'probability')

most positive review

In [236]:
diaper_champ_reviews.sort('predicted_sentiment',ascending=False)[0]

{'name': 'Baby Trend Diaper Champ',
 'review': "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.",
 'rating': 4.0,
 'word_count': {'key': 1.0,
  'have': 1.0,
  'pieces': 1.0,
  'between': 1.0,
  'cushioned': 1.0,
  'piece': 1.0,
  'take': 1.0,
  'overloading': 1.0,
  'be': 1.0,
  'looking': 1.0,
  're': 1.0,
  'stinkiness': 1.0,
  'retain': 1.0,
  'now': 1.0,
  'wonderful': 1.0,
  'worked': 1.0,
  '16': 1.0,
  'and': 3.0,
  'months': 1.0,
  've': 1.0,
  'in': 1.0,
  'i': 3.0,
  'experienced': 1.0,
  'read': 1.0,
 

# prediction of selected words model for most positive review of all words model

In [237]:
diaper_champ_reviews.sort('predicted_sentiment',ascending=False)[0]['predicted_sentiment_sel_words']

0.7925624004659633

Thus, all words model predicts sentiment as 0.999 while selected words only 0.792